In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy 

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-33.57101376132677 159.27564924567685
-46.668873651017265 -159.8862791411141
11.611150714069169 -57.321291599374106
37.85387315182264 144.21814451855346
39.91468523742179 -25.141674690624455
30.917695249580078 -124.13026724094051
68.51093041059795 -178.88463195678403
-38.04080209692269 114.78016248447841
-34.801196846011905 133.269123505442
29.108654963038703 78.82446118380386
-38.69354332537988 10.007297391497673
-51.49686035138035 -53.42973555971983
44.556308715380936 89.9891948041161
74.41843180749507 146.77394074107593
-59.39920585633638 48.4654331166638
30.191718808437074 25.972167700010885
-62.90134350926686 -133.91192401560477
25.830019385486224 -66.42272740780724
79.57498448083095 -33.25828273608403
-16.072961303029274 -170.64457273557068
11.88458322779691 21.060269410605258
87.6404108986311 50.79017661883276
-29.450801288196324 94.24080399200301
-47.12854925940237 -107.88617834658653
46.8506384683742 -19.63199829191612
-53.52896121580131 101.92490479815297
-33.68962226147149 5

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

629

In [6]:
cities = cities[:5]

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [15]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [9]:
# Import the time library and the datetime module from the datetime library
import time
from datetime import datetime

In [10]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | port macquarie
Processing Record 2 of Set 1 | avarua
Processing Record 3 of Set 1 | oistins
Processing Record 4 of Set 1 | kamaishi
Processing Record 5 of Set 1 | rabo de peixe
-----------------------------
Data Retrieval Complete      
-----------------------------


In [11]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Macquarie,-31.4333,152.9167,69.89,76,6,8.19,AU,2023-03-06 18:53:28
1,Avarua,-21.2078,-159.7750,84.25,84,75,17.27,CK,2023-03-06 18:53:28
2,Oistins,13.0667,-59.5333,84.29,61,40,14.97,BB,2023-03-06 18:53:29
3,Kamaishi,39.2667,141.8833,39.52,57,0,6.71,JP,2023-03-06 18:53:30
4,Rabo De Peixe,37.8000,-25.5833,60.31,94,75,18.41,PT,2023-03-06 18:53:30


In [12]:
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [13]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Port Macquarie,AU,2023-03-06 18:53:28,-31.4333,152.9167,69.89,76,6,8.19
1,Avarua,CK,2023-03-06 18:53:28,-21.2078,-159.7750,84.25,84,75,17.27
2,Oistins,BB,2023-03-06 18:53:29,13.0667,-59.5333,84.29,61,40,14.97
3,Kamaishi,JP,2023-03-06 18:53:30,39.2667,141.8833,39.52,57,0,6.71
4,Rabo De Peixe,PT,2023-03-06 18:53:30,37.8000,-25.5833,60.31,94,75,18.41


In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")